In [1]:
# stp 1 Dependecis instal
#!pip install transformers==4.41.2
# Important to install that transformer version, unles it occur error
#See link: https://github.com/THUDM/CogVLM2/issues/181#issuecomment-2381807778
# !pip install transformers sentencepiece datasets evaluate
# !pip install llmlingua

In [2]:
import torch
import json
import time
import os
from transformers import AutoModel, AutoTokenizer, GenerationConfig
from datasets import load_dataset
from tqdm.notebook import tqdm 
import numpy as np
from sklearn.metrics import f1_score
#from LLMLingua.experiments.llmlingua2.evaluation.metrics import qa_f1_score
from metrics import qa_f1_score, f1_score
if 'model' in globals():
    del model 
torch.cuda.empty_cache()

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#######################################
print('cuda is',torch.cuda.is_available()) 
#################################################
# 3 model & dataset 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cuda is True


In [3]:
print('cuda is',torch.cuda.is_available()) 

cuda is True


In [4]:
# ChatGLM-6B
model = AutoModel.from_pretrained(
    "THUDM/chatglm2-6b-32k",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    use_cache=True
).to(device)

model=model.eval()
#print('model config is:',model.generation_config)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b-32k", trust_remote_code=True)#.to(device)

In [6]:
# Monkey-patch the tokenizer's _pad method to handle the padding_side argument
original_pad = tokenizer._pad
def new_pad(self, encoded_inputs, max_length=None, padding_strategy="longest", pad_to_multiple_of=None, return_attention_mask=None, **kwargs):
    return original_pad(encoded_inputs, max_length, padding_strategy, pad_to_multiple_of, return_attention_mask)
tokenizer._pad = new_pad.__get__(tokenizer, type(tokenizer))


In [7]:
# Load the  four datasets
datasets = ["multifieldqa_en", "hotpotqa", "triviaqa", "narrativeqa"]

In [8]:
# LINGUA COMPRESSION

from llmlingua import PromptCompressor
compressor = PromptCompressor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
    device_map="cuda",  # Use GPU
    use_llmlingua2=True,
)

/home/seongjae/miniconda3/envs/seongjae_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def evaluate_multifield(data, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []

    #for idx in range(20): 
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]
        compressed_context = compressor.compress_prompt(
            context,
            rate=ratio,           # Keep 50% tokens (compression ratio)
        )["compressed_prompt"]
        prompt = f"Read the following text and answer briefly.\n\n{compressed_context}\n\nNow, answer the following question based on the above text, only give me the answer and do not output any other words.\n\nQuestion: {question}\nAnswer:"
        torch.cuda.reset_peak_memory_stats()
        start_time = time.time()
        response, _ = model.chat(tokenizer, prompt)
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        peak_memory_usages.append(peak_memory)
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [ ]:
def evaluate_hotpot(data, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []

    #for idx in range(20): 
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]
        compressed_context = compressor.compress_prompt(
            context,
            rate=ratio,           # Keep 50% tokens (compression ratio)
        )["compressed_prompt"]
        prompt = f"Answer the question based on the given passages. Only give me the answer and do not output any other words.\n\nThe following are given passages.\n{compressed_context}\n\nAnswer the question based on the given passages. Only give me the answer and do not output any other words.\n\nQuestion: {question}\nAnswer:"
        torch.cuda.reset_peak_memory_stats()
        start_time = time.time()
        response, _ = model.chat(tokenizer, prompt)
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        peak_memory_usages.append(peak_memory)
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [11]:
def evaluate_trivia(data, ratio=None):
    f1_scores = []
    latencies = []
    memory_usages = []
    peak_memory_usages = []

    #for idx in range(20): 
    for idx in range(len(data)):  # Test on all examples
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers =example["answers"][0]
        compressed_context = compressor.compress_prompt(
            context,
            rate=ratio,           # Keep 50% tokens (compression ratio)
        )["compressed_prompt"]
        prompt = f"Answer the question based on the given passage. Only give me the answer and do not output any other words. The following are some examples.\n\n{compressed_context}\n\n{question}"
        torch.cuda.reset_peak_memory_stats()
        start_time = time.time()
        response, _ = model.chat(tokenizer, prompt)
        latency = time.time() - start_time
        
        # Measure memory
        memory = torch.cuda.memory_allocated() / (1024 ** 2)  # MB
        
        #Measure peak memory?
        peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
        
        # Calculate F1 score
        f1 = qa_f1_score(prediction=response, ground_truth=answers)
        f1_scores.append(f1)
        latencies.append(latency)
        peak_memory_usages.append(peak_memory)
    
    return {
        "avg_f1": np.mean(f1_scores),
        "avg_latency": np.mean(latencies),
        "avg_peak_memory": np.mean(peak_memory_usages)
    }

In [26]:
def evaluate_narrative(data, ratio=None):
    f1_scores = []
    latencies = []
    peak_memory_usages = []
    
    # Model constraints
    max_context_length = 32768  # ChatGLM2-6B-32k's actual limit
    max_new_tokens = 256        # Reserve space for generation
    max_input_length = max_context_length - max_new_tokens  # 32512

    for idx in range(len(data)):
        example = data[idx] 
        context = example["context"]
        question = example["input"]
        answers = example["answers"][0]
        
        # Convert answer to string if needed
        if isinstance(answers, list):
            answers = " ".join(answers)
        answers = str(answers)

        try:
            # 1. Compress context first
            if ratio is not None:
                compressed_context = compressor.compress_prompt(
                    context,
                    rate=ratio,
                    target_token=int(max_input_length * 0.9)  # Leave room for prompt template
                )["compressed_prompt"]
            else:
                compressed_context = context  # No compression

            # 2. Build full prompt
            prompt = f"You are given a story...\n\nStory: {compressed_context}\n\nQuestion: {question}\n\nAnswer:"
            
            # 3. Tokenize and truncate the ENTIRE prompt
            inputs = tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=max_input_length
            ).to(model.device)

            # 4. Generate with length control
            torch.cuda.empty_cache()
            torch.cuda.reset_peak_memory_stats()
            start_time = time.time()
            
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.01
            )
            
            latency = time.time() - start_time
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # 5. Calculate metrics
            peak_memory = torch.cuda.max_memory_allocated() / (1024 ** 2)
            f1 = qa_f1_score(response.split("Answer:")[-1].strip(), answers)
            
            f1_scores.append(f1)
            latencies.append(latency)
            peak_memory_usages.append(peak_memory)

        except Exception as e:
            print(f"Error processing example {idx}: {str(e)}")
            continue
    
    return {
        "avg_f1": np.mean(f1_scores) if f1_scores else 0,
        "avg_latency": np.mean(latencies) if latencies else 0,
        "avg_peak_memory": np.mean(peak_memory_usages) if peak_memory_usages else 0
    }

In [ ]:
# Reusable function to print and save results
        
def log_result_LLMLingua(message):
    #print(message)
    with open("results_LLMLingua.txt", "a") as f:  # 'a' = append mode
        f.write(message + "\n")

In [14]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[0]  #multifiledqa_en
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
for ratio in pruning_ratios:
    # lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    lingua_results = evaluate_multifield(data, ratio=ratio)
    
    print(f"\n==={dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n===20 samples {dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")

Token indices sequence length is longer than the specified maximum sequence length for this model (1237 > 512). Running this sequence through the model will result in indexing errors



===multifieldqa_en  llmlingua compression (25%) ===
F1: 0.19, Latency: 2.47s,  Peak Memory: 14417.95 MB

===multifieldqa_en  llmlingua compression (50%) ===
F1: 0.27, Latency: 17.86s,  Peak Memory: 14752.85 MB

===multifieldqa_en  llmlingua compression (75%) ===
F1: 0.38, Latency: 1.41s,  Peak Memory: 15072.73 MB


In [15]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[1]  #hotpotqa
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
for ratio in pruning_ratios:
    # lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    lingua_results = evaluate_hotpot(data, ratio=ratio)
    
    print(f"\n==={dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n===20 samples {dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")


===hotpotqa  llmlingua compression (25%) ===
F1: 0.30, Latency: 0.70s,  Peak Memory: 14749.06 MB

===hotpotqa  llmlingua compression (50%) ===
F1: 0.35, Latency: 1.25s,  Peak Memory: 15346.20 MB

===hotpotqa  llmlingua compression (75%) ===
F1: 0.38, Latency: 1.92s,  Peak Memory: 15934.34 MB


In [16]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[2]  #triviaqa
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
for ratio in pruning_ratios:
    # lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    lingua_results = evaluate_trivia(data, ratio=ratio)
    
    print(f"\n==={dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n===20 samples {dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")


===triviaqa  llmlingua compression (25%) ===
F1: 0.42, Latency: 0.70s,  Peak Memory: 14769.61 MB

===triviaqa  llmlingua compression (50%) ===
F1: 0.40, Latency: 1.20s,  Peak Memory: 15281.46 MB

===triviaqa  llmlingua compression (75%) ===
F1: 0.41, Latency: 1.78s,  Peak Memory: 15789.83 MB


In [27]:
pruning_ratios = [0.25, 0.5, 0.75]
# for dataset_name in datasets:
dataset_name = datasets[3]  #narrativeqa
data = load_dataset("THUDM/LongBench", dataset_name, split="test", cache_dir="custom_cache_dir")
for ratio in pruning_ratios:
    # lingua_results = evaluate(data, compress_with_llmlingua, ratio=ratio)
    lingua_results = evaluate_narrative(data, ratio=ratio)
    
    print(f"\n==={dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    print(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")
    log_result_LLMLingua(f"\n===20 samples {dataset_name}  llmlingua compression ({int(ratio*100)}%) ===")
    log_result_LLMLingua(f"F1: {lingua_results['avg_f1']:.2f}, Latency: {lingua_results['avg_latency']:.2f}s,  Peak Memory: {lingua_results['avg_peak_memory']:.2f} MB")

/home/seongjae/miniconda3/envs/seongjae_env/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



===narrativeqa  llmlingua compression (25%) ===
F1: 0.21, Latency: 9.47s,  Peak Memory: 17819.50 MB

===narrativeqa  llmlingua compression (50%) ===
F1: 0.21, Latency: 9.68s,  Peak Memory: 17819.39 MB

===narrativeqa  llmlingua compression (75%) ===
F1: 0.21, Latency: 9.04s,  Peak Memory: 17819.39 MB
